In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ast import literal_eval
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder, MultiLabelBinarizer, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import MeanSquaredError, Accuracy
from tensorflow.keras import models
from tensorflow.keras.layers import Dense, Concatenate, Input, Dropout,  LSTM, Flatten
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.regularizers import l1_l2
from keras.models import Sequential
import nltk
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize


In [2]:
df = pd.read_csv('/Users/louishagenbucher/Documents/GitHub/mtg_nlp_price_prediction/data/clean_cards.csv')
df.sample(5)

,cmc,colors,legendary,types,sub_types,oracle_text,keywords,power,toughness,rarity,reserved,released_at,booster,usd
9390,2,['W'],False,['Enchantment'],['none'],"At the beginning of your end step, if you gain...",['no keywords'],0,0,uncommon,False,9711,True,0.05
15773,3,['W'],False,['Creature'],['Pegasus'],Flying (This creature can't be blocked except ...,"['Flying', 'First strike']",2,1,common,False,4258,True,0.42
5961,3,['U'],False,['Creature'],['Spirit'],Flying\nAt the beginning of combat on your tur...,['Flying'],3,1,rare,False,10215,True,0.24
19098,1,['W'],False,['Creature'],"['Human', 'Monk']",Lifelink (Damage dealt by this creature also c...,['Lifelink'],1,1,rare,False,8752,True,18.82
11354,6,['G'],False,['Sorcery'],['none'],Each creature you control gets +3/+3 until end...,['no keywords'],0,0,uncommon,False,7898,True,0.05


In [3]:
no_prices_df = df[df['usd'].isna()]

In [4]:
df.drop(df[df['usd'].isna()].index, inplace=True)
df.reset_index(drop=True, inplace=True)

In [5]:
df['colors'] = df['colors'].apply(literal_eval)

In [6]:
mlb = MultiLabelBinarizer()
df= df.join(pd.DataFrame(mlb.fit_transform(df['colors']),
                                      columns=mlb.classes_,
                                      index=df.index))
legend_lb= LabelBinarizer()
df = df.join(pd.DataFrame(legend_lb.fit_transform(df['legendary']), columns=['legendary'])
             .add_prefix('_'))
booster_lb= LabelBinarizer()
df = df.join(pd.DataFrame(legend_lb.fit_transform(df['booster']), columns=['booster'])
             .add_prefix('_'))
reserved_lb= LabelBinarizer()
df = df.join(pd.DataFrame(legend_lb.fit_transform(df['reserved']), columns=['resrved'])
             .add_prefix('_'))

df = df.drop(['legendary', 'booster', 'reserved', 'colors'], axis=1)
df.head(3)

,cmc,types,sub_types,oracle_text,keywords,power,toughness,rarity,released_at,usd,B,C,G,N,R,U,W,_legendary,_booster,_resrved
0,2,['Artifact'],['Equipment'],Equipped creature gets +2/+2.\nEquip {3} ({3}:...,['Equip'],0,0,common,10096,0.02,0,0,0,0,0,0,1,0,1,0
1,4,['Creature'],"['Bird', 'Rogue']",Flying\nWhen Aarakocra Sneak enters the battle...,['Flying'],1,4,common,10418,0.06,0,0,0,0,0,1,0,0,1,0
2,5,['Creature'],"['Astartes', 'Warrior']",Trample\nMark of Chaos Ascendant — During your...,"['Mark of Chaos Ascendant', 'Trample']",5,5,mythic,10537,2.81,1,0,0,0,1,1,0,1,0,0


#### Text processing

In [7]:
df['tokenized_text']=df['oracle_text'].apply(word_tokenize)

In [8]:
text_model = Word2Vec(sentences=df['tokenized_text'], vector_size=100, window=5, min_count=1, workers=4)

In [9]:
def get_sentence_embedding(tokens):
    tokens = [token for token in tokens if token in text_model.wv.key_to_index]
    if tokens:
        return sum(text_model.wv[token] for token in tokens) / len(tokens)
    else:
        return None

In [10]:
df['text_embedding'] = df['oracle_text'].apply(get_sentence_embedding)

In [11]:
df['text_embedding']

0        [0.074369654, 0.30684194, -0.017265068, 1.0200...
1        [-0.4148831, -0.09660204, -0.051159177, 0.9739...
2        [-0.4004118, 0.039231997, -0.1324655, 0.632887...
3        [-0.18434682, 0.42023063, 0.04573707, 1.022404...
4        [-0.5161276, -0.11405305, -0.035779774, 0.8323...
                               ...                        
25484    [-0.34699845, -0.0676399, -0.12240102, 0.84550...
25485    [-0.5216625, 0.029419824, -0.063910775, 1.1866...
25486    [-0.45051646, -0.13936377, -0.040107027, 0.895...
25487    [-0.47539547, -0.1498929, -0.12021196, 0.91447...
25488    [-0.33509842, 0.146706, -0.03125711, 0.8986092...
Name: text_embedding, Length: 25489, dtype: object

In [12]:
df['tokenized_types']=df['types'].apply(word_tokenize)
types_model = Word2Vec(sentences=df['types'], vector_size=100, window=5, min_count=1, workers=4)
def get_types_embedding(tokens):
    tokens = [token for token in tokens if token in types_model.wv.key_to_index]
    if tokens:
        return sum(types_model.wv[token] for token in tokens) / len(tokens)
    else:
        return None
df['types_embedding'] = df['types'].apply(get_types_embedding)
df['types_embedding']

0        [0.09025344, -0.0933645, 0.040706877, 0.102777...
1        [-0.23466605, 0.054150928, -0.23474295, -0.053...
2        [-0.23466605, 0.054150928, -0.23474295, -0.053...
3        [0.37246984, -0.10229263, 0.006371121, -0.4545...
4        [0.09025344, -0.0933645, 0.040706877, 0.102777...
                               ...                        
25484    [-0.23466605, 0.054150928, -0.23474295, -0.053...
25485    [0.43425027, -0.1511413, -0.17898294, -0.50388...
25486    [-0.23466605, 0.054150928, -0.23474295, -0.053...
25487    [-0.23466605, 0.054150928, -0.23474295, -0.053...
25488    [-0.23466605, 0.054150928, -0.23474295, -0.053...
Name: types_embedding, Length: 25489, dtype: object

In [13]:
df['tokenized_subtypes']=df['sub_types'].apply(word_tokenize)
subtypes_model = Word2Vec(sentences=df['sub_types'], vector_size=100, window=5, min_count=1, workers=4)
def get_types_embedding(tokens):
    tokens = [token for token in tokens if token in subtypes_model.wv.key_to_index]
    if tokens:
        return sum(subtypes_model.wv[token] for token in tokens) / len(tokens)
    else:
        return None
df['subtypes_embedding'] = df['sub_types'].apply(get_types_embedding)
df['subtypes_embedding']

0        [-0.433625, 0.34098607, 0.02097249, -0.3161194...
1        [-0.29299492, 0.032801047, 0.19450957, -0.2610...
2        [-0.16101502, -0.10935294, 0.10891135, -0.2571...
3        [-0.2388947, 0.05914274, 0.07720654, -0.206427...
4        [-0.2388947, 0.05914274, 0.07720654, -0.206427...
                               ...                        
25484    [-0.25974986, -0.08754499, 0.24316484, -0.2755...
25485    [-0.2388947, 0.05914274, 0.07720654, -0.206427...
25486    [-0.4354455, 0.08101188, 0.18476349, -0.253234...
25487    [-0.22736448, 0.029368427, 0.2064418, -0.17124...
25488    [-0.26826623, 0.0052392553, 0.10771507, -0.329...
Name: subtypes_embedding, Length: 25489, dtype: object

In [14]:
df['tokenized_keywords']=df['keywords'].apply(word_tokenize)
keywords_model = Word2Vec(sentences=df['keywords'], vector_size=100, window=5, min_count=1, workers=4)
def get_types_embedding(tokens):
    tokens = [token for token in tokens if token in keywords_model.wv.key_to_index]
    if tokens:
        return sum(keywords_model.wv[token] for token in tokens) / len(tokens)
    else:
        return None
df['keywords_embedding'] = df['keywords'].apply(get_types_embedding)
df['keywords_embedding']

0        [-0.03499625, -0.14067245, 0.27548707, -0.3367...
1        [0.05384749, -0.15204267, -0.18976814, -0.1325...
2        [0.14801234, -0.1327785, 0.017358556, -0.02545...
3        [-0.27851194, 0.047724154, 0.023241542, 0.1952...
4        [-0.27851194, 0.047724154, 0.023241542, 0.1952...
                               ...                        
25484    [-0.010021531, -0.19710003, 0.14082009, -0.121...
25485    [-0.27851194, 0.047724154, 0.023241542, 0.1952...
25486    [0.05384749, -0.15204267, -0.18976814, -0.1325...
25487    [-0.27851194, 0.047724154, 0.023241542, 0.1952...
25488    [-0.15589608, -0.032055657, 0.002933483, 0.068...
Name: keywords_embedding, Length: 25489, dtype: object

In [15]:
df.columns

Index(['cmc', 'types', 'sub_types', 'oracle_text', 'keywords', 'power',
       'toughness', 'rarity', 'released_at', 'usd', 'B', 'C', 'G', 'N', 'R',
       'U', 'W', '_legendary', '_booster', '_resrved', 'tokenized_text',
       'text_embedding', 'tokenized_types', 'types_embedding',
       'tokenized_subtypes', 'subtypes_embedding', 'tokenized_keywords',
       'keywords_embedding'],
      dtype='object')

In [16]:
df.drop(columns=['sub_types','types', 'keywords', 'oracle_text',
        'tokenized_types','tokenized_subtypes', 'tokenized_keywords',
        'tokenized_text'],  inplace=True)

#### The Pipeline and neural network

In [17]:
df

,cmc,power,toughness,rarity,released_at,usd,B,C,G,N,R,U,W,_legendary,_booster,_resrved,text_embedding,types_embedding,subtypes_embedding,keywords_embedding
0,2,0,0,common,10096,0.02,0,0,0,0,0,0,1,0,1,0,"[0.074369654, 0.30684194, -0.017265068, 1.0200...","[0.09025344, -0.0933645, 0.040706877, 0.102777...","[-0.433625, 0.34098607, 0.02097249, -0.3161194...","[-0.03499625, -0.14067245, 0.27548707, -0.3367..."
1,4,1,4,common,10418,0.06,0,0,0,0,0,1,0,0,1,0,"[-0.4148831, -0.09660204, -0.051159177, 0.9739...","[-0.23466605, 0.054150928, -0.23474295, -0.053...","[-0.29299492, 0.032801047, 0.19450957, -0.2610...","[0.05384749, -0.15204267, -0.18976814, -0.1325..."
2,5,5,5,mythic,10537,2.81,1,0,0,0,1,1,0,1,0,0,"[-0.4004118, 0.039231997, -0.1324655, 0.632887...","[-0.23466605, 0.054150928, -0.23474295, -0.053...","[-0.16101502, -0.10935294, 0.10891135, -0.2571...","[0.14801234, -0.1327785, 0.017358556, -0.02545..."
3,0,0,0,common,2861,0.14,0,0,0,1,0,0,0,0,1,0,"[-0.18434682, 0.42023063, 0.04573707, 1.022404...","[0.37246984, -0.10229263, 0.006371121, -0.4545...","[-0.2388947, 0.05914274, 0.07720654, -0.206427...","[-0.27851194, 0.047724154, 0.023241542, 0.1952..."
4,3,0,0,rare,9634,0.09,0,1,0,0,0,0,0,0,0,0,"[-0.5161276, -0.11405305, -0.035779774, 0.8323...","[0.09025344, -0.0933645, 0.040706877, 0.102777...","[-0.2388947, 0.05914274, 0.07720654, -0.206427...","[-0.27851194, 0.047724154, 0.023241542, 0.1952..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25484,5,7,2,mythic,7604,0.42,1,0,0,0,1,0,1,1,1,0,"[-0.34699845, -0.0676399, -0.12240102, 0.84550...","[-0.23466605, 0.054150928, -0.23474295, -0.053...","[-0.25974986, -0.08754499, 0.24316484, -0.2755...","[-0.010021531, -0.19710003, 0.14082009, -0.121..."
25485,4,0,0,rare,4258,0.82,0,0,0,0,0,1,0,0,1,0,"[-0.5216625, 0.029419824, -0.063910775, 1.1866...","[0.43425027, -0.1511413, -0.17898294, -0.50388...","[-0.2388947, 0.05914274, 0.07720654, -0.206427...","[-0.27851194, 0.047724154, 0.023241542, 0.1952..."
25486,4,1,4,rare,10446,0.53,1,0,0,0,0,1,1,1,1,0,"[-0.45051646, -0.13936377, -0.040107027, 0.895...","[-0.23466605, 0.054150928, -0.23474295, -0.053...","[-0.4354455, 0.08101188, 0.18476349, -0.253234...","[0.05384749, -0.15204267, -0.18976814, -0.1325..."
25487,3,2,3,rare,10376,0.25,0,0,0,0,1,0,0,1,0,0,"[-0.47539547, -0.1498929, -0.12021196, 0.91447...","[-0.23466605, 0.054150928, -0.23474295, -0.053...","[-0.22736448, 0.029368427, 0.2064418, -0.17124...","[-0.27851194, 0.047724154, 0.023241542, 0.1952..."


In [18]:
X = df.drop(columns='usd')
y = df['usd']

In [19]:
numerical_features = ['power', 'toughness', 'released_at', 'cmc']
categorical_features = ['rarity']
remaining_features = X.drop(columns=['rarity', 'power', 'toughness', 'released_at', 'cmc'])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ],
    remainder='passthrough'
)

pipeline = make_pipeline(preprocessor)

In [20]:
X_preprocessed= pipeline.fit_transform(X)

In [21]:
column_names = pipeline.get_feature_names_out()
column_names

array(['num__power', 'num__toughness', 'num__released_at', 'num__cmc',
       'cat__rarity_common', 'cat__rarity_mythic', 'cat__rarity_rare',
       'cat__rarity_special', 'cat__rarity_uncommon', 'remainder__B',
       'remainder__C', 'remainder__G', 'remainder__N', 'remainder__R',
       'remainder__U', 'remainder__W', 'remainder___legendary',
       'remainder___booster', 'remainder___resrved',
       'remainder__text_embedding', 'remainder__types_embedding',
       'remainder__subtypes_embedding', 'remainder__keywords_embedding'],
      dtype=object)

In [22]:
processed_df = pd.DataFrame(X_preprocessed, columns=column_names)

In [23]:
processed_df.head(3)

,num__power,num__toughness,num__released_at,num__cmc,cat__rarity_common,cat__rarity_mythic,cat__rarity_rare,cat__rarity_special,cat__rarity_uncommon,remainder__B,...,remainder__R,remainder__U,remainder__W,remainder___legendary,remainder___booster,remainder___resrved,remainder__text_embedding,remainder__types_embedding,remainder__subtypes_embedding,remainder__keywords_embedding
0,-0.787786,-0.818064,0.83164,-0.74473,1.0,0.0,0.0,0.0,0.0,0,...,0,0,1,0,1,0,"[0.074369654, 0.30684194, -0.017265068, 1.0200...","[0.09025344, -0.0933645, 0.040706877, 0.102777...","[-0.433625, 0.34098607, 0.02097249, -0.3161194...","[-0.03499625, -0.14067245, 0.27548707, -0.3367..."
1,-0.237532,1.302998,0.930864,0.397826,1.0,0.0,0.0,0.0,0.0,0,...,0,1,0,0,1,0,"[-0.4148831, -0.09660204, -0.051159177, 0.9739...","[-0.23466605, 0.054150928, -0.23474295, -0.053...","[-0.29299492, 0.032801047, 0.19450957, -0.2610...","[0.05384749, -0.15204267, -0.18976814, -0.1325..."
2,1.963486,1.833263,0.967534,0.969104,0.0,1.0,0.0,0.0,0.0,1,...,1,1,0,1,0,0,"[-0.4004118, 0.039231997, -0.1324655, 0.632887...","[-0.23466605, 0.054150928, -0.23474295, -0.053...","[-0.16101502, -0.10935294, 0.10891135, -0.2571...","[0.14801234, -0.1327785, 0.017358556, -0.02545..."


In [24]:
processed_df =processed_df.astype({
    'num__power': float,
    'num__toughness': float,
    'num__released_at': float,
    'num__cmc': float
})

In [25]:
processed_df = processed_df.astype({
    "cat__rarity_common": int,
    "cat__rarity_mythic": int,
    "cat__rarity_rare": int,
    "cat__rarity_special": int,
    "cat__rarity_uncommon": int,
    "remainder__B": int,
    "remainder__C": int,
    "remainder__G": int,
    "remainder__N": int,
    "remainder__R": int,
    "remainder__U": int,
    "remainder__W": int,
    "remainder___legendary": int,
    "remainder___booster": int,
    "remainder___resrved": int
})

In [26]:
processed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25489 entries, 0 to 25488
Data columns (total 23 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   num__power                     25489 non-null  float64
 1   num__toughness                 25489 non-null  float64
 2   num__released_at               25489 non-null  float64
 3   num__cmc                       25489 non-null  float64
 4   cat__rarity_common             25489 non-null  int64  
 5   cat__rarity_mythic             25489 non-null  int64  
 6   cat__rarity_rare               25489 non-null  int64  
 7   cat__rarity_special            25489 non-null  int64  
 8   cat__rarity_uncommon           25489 non-null  int64  
 9   remainder__B                   25489 non-null  int64  
 10  remainder__C                   25489 non-null  int64  
 11  remainder__G                   25489 non-null  int64  
 12  remainder__N                   25489 non-null 

In [28]:
X_numerical = processed_df[['num__power', 'num__toughness', 'num__released_at', 'num__cmc']]
X_categorical = processed_df[['remainder___legendary', 'remainder___booster', 'remainder___resrved',
                              'cat__rarity_common', 'cat__rarity_mythic', 'cat__rarity_rare', 'cat__rarity_special', 'cat__rarity_uncommon',
                              'remainder__B', 'remainder__C', 'remainder__G', 'remainder__N', 'remainder__R', 'remainder__U', 'remainder__W']]
X_text = processed_df[['remainder__text_embedding']]
X_subtypes= processed_df[['remainder__subtypes_embedding']]
X_types = processed_df[['remainder__types_embedding']]
X_keys = processed_df[['remainder__keywords_embedding']]

In [29]:
X_numerical.shape, X_categorical.shape, X_text.shape, X_subtypes.shape, X_types.shape, X_keys.shape

((25489, 4), (25489, 15), (25489, 1), (25489, 1), (25489, 1), (25489, 1))

In [31]:
X_text['remainder__text_embedding'][0].shape, X_subtypes['remainder__subtypes_embedding'][0].shape, X_types['remainder__types_embedding'][0].shape, X_keys['remainder__keywords_embedding'][0].shape

((100,), (100,), (100,), (100,))

In [ ]:
X_numerical_train, X_numerical_test, X_categorical_train, X_categorical_test, X_text_train, X_text_test,X_subtypes_train, X_subtypes_test, X_types_train, X_types_test, X_keys_train, X_keys_test, y_train, y_test = train_test_split(
    X_numerical, X_categorical, X_text, X_subtypes, X_types, X_keys, y, test_size=0.30, random_state=42
)

In [ ]:
def initialize_model():

    model_numerical = Sequential([
        Input(shape=(4,), name='input_numerical'),
        Dense(32, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(8, activation='relu')
    ], name='model_numerical')

    model_categorical = Sequential([
        Input(shape=(15,), name='input_categorical'),
        Dense(64, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dropout(0.2),
        Dense(16, activation='relu')
    ], name='model_categorical')

    model_text = Sequential([
        Input(shape=(79, 768), name='input_text'),
        LSTM(128, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01), return_sequences=True),
        LSTM(64, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        Dropout(0.2),
        Dense(256, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        Dense(128, activation='relu'),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(256, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
    ], name='model_text')


    model_type = Sequential([
        Input(shape=(11, 768), name='input_type'),
        LSTM(128, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
    ], name='model_type')


    model_subtype = Sequential([
        Input(shape=(15, 768), name='input_subtype'),
        LSTM(128, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(32, activation='relu')
    ], name='model_subtype')


    model_keys = Sequential([
        Input(shape=(19, 768), name='input_keys'),
        LSTM(128, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(32, activation='relu')
    ], name='model_keys')


    # Concatenate the outputs of all Sequential models
    concatenated = Concatenate()([
        model_numerical.output,
        model_categorical.output,
        model_text.output,
        model_type.output,
        model_keys.output,
        model_subtype.output
    ])

    output = Dense(1, activation='linear', name='output')(concatenated)


    model = models.Model(inputs=[model_numerical.input, model_categorical.input, model_text.input, model_type.input, model_keys.input, model_subtype.input], outputs=output)

    model.compile(loss='mean_squared_error',
                  optimizer='adam',
                  metrics=['mse', 'accuracy'])

    return model


In [ ]:
def get_history():
    es = EarlyStopping(patience=10, restore_best_weights=False)

    train_data = {
        'input_numerical': X_numerical_train,
        'input_categorical': X_categorical_train,
        'input_text': X_text_train,
        'input_type': X_types_train,
        'input_subtype': X_subtypes_train,
        'input_keys': X_keys_train
    }

    test_data = {
        'input_numerical': X_numerical_test,
        'input_categorical': X_categorical_test,
        'input_text': X_text_test,
        'input_type': X_types_test,
        'input_subtype': X_subtypes_test,
        'input_keys': X_keys_test
    }

    history = model.fit(
        train_data,
        y_train,
        epochs=20,
        batch_size=16,
        validation_data=(test_data, y_test),
        callbacks=[es],
        verbose=1
    )

    return history


In [ ]:
model = initialize_model()

In [ ]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_text (InputLayer)     [(None, 79, 768)]            0         []                            
                                                                                                  
 lstm_10 (LSTM)              (None, 79, 128)              459264    ['input_text[0][0]']          
                                                                                                  
 lstm_11 (LSTM)              (None, 64)                   49408     ['lstm_10[0][0]']             
                                                                                                  
 dropout_41 (Dropout)        (None, 64)                   0         ['lstm_11[0][0]']             
                                                                                            

In [ ]:
get_history()

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type tensorflow.python.framework.ops.EagerTensor).

In [ ]:
test_data = {
        'input_numerical': X_numerical_test,
        'input_categorical': X_categorical_test,
        'input_text': X_text_test,
        'input_type': X_types_test,
        'input_subtype': X_subtypes_test,
        'input_keys': X_keys_test
    }
model.evaluate(test_data, y_test)

239/239 [==============================] - 0s 2ms/step - loss: 1885.5562 - mse: 1881.6256 - accuracy: 2.6154e-04


[1885.55615234375, 1881.6256103515625, 0.00026154046645388007]

In [ ]:
no_prices_df = df[df['usd'].isna()]